In [ ]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# PyTorch
import torch
#import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch import FloatTensor, LongTensor

from torchvision.models import resnet50   # resnet34


class ResNet50Network(nn.Module):
    def __init__(self, output_size, no_columns):
        super().__init__()
        self.no_columns, self.output_size = no_columns, output_size
        
        # Define Feature part (IMAGE)
        self.features = resnet50(pretrained=True) # 1000 neurons out
        # (metadata)
        self.csv = nn.Sequential(nn.Linear(self.no_columns, 500),
                                 nn.BatchNorm1d(500),
                                 nn.ReLU(),
                                 nn.Dropout(p=0.2))
        
        # Define Classification part
        self.classification = nn.Linear(1000 + 500, output_size)
        
        
    def forward(self, image, csvdata, prints=False):
        if prints: print('Input Image shape:', image.shape, '\n'+
                         'Input csvdata shape:', csvdata.shape)
        
        # Image CNN
        image = self.features(image)
        if prints: print('Features Image shape:', image.shape)
        
        # CSV FNN
        csvdata = self.csv(csvdata)
        if prints: print('CSV Data:', csvdata.shape)
            
        # Concatenate layers from image with layers from csv_data
        image_csv_data = torch.cat((image, csvdata), dim=1)
        if prints: print('Concatenated Data:', image_csv_data.shape)
        
        # CLASSIF
        out = self.classification(image_csv_data)
        if prints: print('Out shape:', out.shape)
        
        return out

